In [1]:
import gpxpy
import pandas as pd

import matplotlib.pyplot as plt

import gzip
from lxml import etree

import folium


/Users/bac/Documents/PYTHON_PROJECTS/Strava_Analysis/strava_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
main_dir = 'STRAVA+export_8029714/activities'

## Generate a dataframe

In [ ]:

# Load a GPX file
with open(f'{main_dir}/261971610.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)


In [ ]:
points = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevation': point.elevation,
                'time': point.time
            })

df = pd.DataFrame(points)

In [ ]:
df

## Generate a Map

In [ ]:

# Get the central point of the route to center the map
start_coords = (df['latitude'].mean(), df['longitude'].mean())

# Create the map
m = folium.Map(location=start_coords, zoom_start=13)

# Draw the route as a line
folium.PolyLine(
    list(zip(df['latitude'], df['longitude'])),
    color='blue',
    weight=4,
    opacity=0.8
).add_to(m)

# Optionally, mark start and end points
folium.Marker(
    location=[df.iloc[0]['latitude'], df.iloc[0]['longitude']],
    popup='Start',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[df.iloc[-1]['latitude'], df.iloc[-1]['longitude']],
    popup='End',
    icon=folium.Icon(color='red')
).add_to(m)

# Show the map in Jupyter or export to HTML
m.save("route_map.html")

## Generate elevation map

In [ ]:

# Optional: calculate distance for x-axis (if time/distance based is preferred)
df['time'] = pd.to_datetime(df['time'])
df['elapsed_time_min'] = (df['time'] - df['time'].min()).dt.total_seconds() / 60

# Plot elevation over time
plt.figure(figsize=(12, 5))
plt.plot(df['elapsed_time_min'], df['elevation'], color='darkorange')
plt.xlabel('Time Elapsed (minutes)')
plt.ylabel('Elevation (m)')
plt.title('Elevation Profile')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

with gzip.open('activity.tcx.gz', 'rb') as f:
    tree = etree.parse(f)
    root = tree.getroot()

    for elem in root.iter():
        print(elem.tag, elem.text)
